# Random Forest Regression - Multi-node, Multi-GPU

Note: The tools used in this demo are still classified as experimental by the RAPIDS-AI team. 


RAPIDS leverages Dask to do embarrassingly-parallel model fitting. For a random forest with N trees fit by K workers, each worker will build and fit N / K trees. 

In order to build an accurate RF regressor, it is necessary to ensure that each worker has a representitive partition of the data. One route would be to evenly distribute properly-shuffled data. If the GPU cluster has enough working memory, the caller may replicate the entirety of the data to all workers. 

Look [here](https://docs.rapids.ai/api/cuml/stable/api.html#cuml.dask.ensemble.RandomForestRegressor) for more informationr regarding distributed RF regressors.

In [ ]:
import numpy as np
import sklearn

import pandas as pd
import cudf
import cuml

from sklearn.metrics import mean_absolute_error, r2_score
from sklearn import model_selection

from cuml.dask.common import utils as dask_utils
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import dask_cudf

from cuml.dask.ensemble import RandomForestRegressor as cumlDaskRF
from sklearn.ensemble import RandomForestRegressor as sklRF

!nvidia-smi

## Dask Cluster

Here Dask forms its own local "cluster", which will use all GPUs on the local host by default. 

In [ ]:
cluster = LocalCUDACluster(threads_per_worker=1)
c = Client(cluster)

workers = c.has_what().keys()
n_workers = len(workers)
n_streams = 8

## Define parameters

Here we will set parameters for our random forest building

In [ ]:
max_depth = 80
n_bins = 16
n_trees = 1000

## Import data

In [ ]:
%%time

#df_pandas = pd.read_csv('Data/train_validate_10_final.csv').astype(np.float32)
df_pandas = pd.read_csv('Data/aviris_bands_extract_final.csv').astype(np.float32) # Read the csv into a pandas DataFrame

cpu_training_predictors = df_pandas['depth_m']
cpu_covariates = df_pandas.drop(['depth_m'], axis=1)

X_train, X_test, y_train, y_test = model_selection.train_test_split(
                                                            cpu_covariates,
                                                            cpu_training_predictors,
                                                            shuffle=True,
                                                            train_size=0.8)

In [ ]:
n_partitions = n_workers

def distribute(X, y):
    X_cudf = cudf.DataFrame.from_pandas(pd.DataFrame(X))
    y_cudf = cudf.Series(y)
    
    #Partition with Dask
    # Workers will train on 1/n_partitions of the data
    X_dask = dask_cudf.from_cudf(X_cudf, npartitions=n_partitions)
    y_dask = dask_cudf.from_cudf(y_cudf, npartitions=n_partitions)
    
    # Persis to cache data in active memory
    X_dask, y_dask = \
        dask_utils.persist_across_workers(c, [X_dask, y_dask], workers=workers)
    
    return X_dask, y_dask

X_train_dask, y_train_dask = distribute(X_train, y_train)
X_test_dask, y_test_dask = distribute(X_test, y_test)

## Scikit-Learn model (for comparison)

Sci-kit does offer multi-CPU support via joblib.

Note: Be wary that if using the large 390-band dataset, an sklearn model may take up to 30 minutes to fit. 

In [ ]:
%%time

skl_model = sklRF(max_depth=max_depth, n_estimators=n_trees, n_jobs=-1)
skl_model.fit(X_train, y_train)

## Train the Dask-cuML model

In [ ]:
%%time

cuml_model = cumlDaskRF(max_depth=max_depth, n_estimators=n_trees, n_bins=n_bins, n_streams=n_streams)
cuml_model.fit(X_train_dask, y_train_dask)

wait(cuml_model.rfs) # Allow async tasks to finish

## Predict and measure accuracy

In [ ]:
skl_y_pred = skl_model.predict(X_test)
cuml_y_pred = cuml_model.predict(X_test_dask).compute().to_array()

print("SKLearn mean_absolute_erroror: ", mean_absolute_error(y_test, skl_y_pred))
print("CuML mean_absolute_error: ", mean_absolute_error(y_test, cuml_y_pred))

print("SKLearn r^2: ", r2_score(y_test, skl_y_pred))
print("CuML r^2: ", r2_score(y_test, cuml_y_pred))

As you may notice, the r^2 score for the cuML-Dask model is negative. This is actually a positive score. This happens when the cuML predictions are measured against host-bound data. This is believed to be due to how data is store/represented in GPUs vs CPUs.